In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# DONE update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 30039
db = 'AAC'
col = 'animals'
db = AnimalShelter(username, password, host, port, db, col)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read())

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#DONE Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#DONE ME Place the HTML image tag in the line below into the app.layout code according to your design
#DONE ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([

    
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(style={'display': 'flex', 'flex-direction': 'row', 'align-items': 'center', 'justify-content': 'center', 'min-height': '200px'},children=[
        html.B(html.H1('CS-340 Dashboard - Fellipe de Moraes',style={'vertical-align':'middle','display':'inline','margin':'1em','vertical-align':'middle'})),
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'width': '150px',
                    'height': 'auto',
                    'border': '2px solid blue',
                    'borderRadius': '10px',
                    'boxShadow': '5px 5px 10px lightgray'
                }),
            href='https://www.snhu.edu',
            target='_blank',
            title='SNHU')]
       
        ),
    html.Hr(),
    html.Div(
        [
            dcc.RadioItems(
                id='filter-type',  # ADD AN ID
                options=[
                    {'label': 'Water rescue', 'value': 'water'},
                    {'label': 'Wilderness rescue', 'value': 'wilderness'},
                    {'label': 'Disaster rescue', 'value': 'disaster'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                inline=True,
                value='reset'  # Optionally set an initial value or leave as None
            )        
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                             columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                             data=df.to_dict('records'),
#DONE: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                            page_current=0,  # Start on page 1 (0-indexed)
                            page_size=10,   # Display 10 rows per page
                            page_action='native',  # Enable native pagination
                            filter_action='native', # Enable filtering
                            sort_action='native',   # Enable sorting
                            sort_mode='multi',      # Allow multi-column sorting
                            column_selectable=False,  # Allow single column selection
                            row_selectable='single',     # Allow multiple row selection
                            row_deletable=False,      # Prevent row deletion by default
                            selected_rows=[0],
                            selected_columns=[],
                            style_header={          # Styling for the header row
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read(water=True))
    elif filter_type == 'wilderness':
        df = pd.DataFrame.from_records(db.read(wild=True))
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(db.read(disaster=True))
    elif filter_type == 'reset':
        df = pd.DataFrame.from_records(db.read())
    df.drop(columns=['_id'],inplace=True)

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
       
    return data
    

        



# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', "value")])
def update_graphs(viewData, radio):
    if viewData is None or not viewData:
        # Handle the case where there is no data in the table
        return [dcc.Graph(figure=px.pie(pd.DataFrame(), names=[], title='No Data'))]

    # Create a new DataFrame from the derived virtual data
    df_view = pd.DataFrame(viewData)

    # Check if the 'breed' column exists in the derived data
    if 'breed' not in df_view.columns:
        return [dcc.Graph(figure=px.pie(pd.DataFrame(), names=[], title='Breed Data Missing'))]
    
    if radio == 'reset':
        return [dcc.Graph(figure=px.pie(pd.DataFrame(), names=[], title='Select a filter'))]

    # Create the pie chart using the derived data
    fig = px.pie(df_view, names='breed', title='Preferred Animals')

    return [dcc.Graph(figure=fig)]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.61,-97.48], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:15085/


/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1073: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

